In [13]:
!pip install sqlalchemy sqlite-utils

In [14]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from google.colab import files

In [15]:
print("Upload your dataset CSV:")
uploaded = files.upload()
filename = list(uploaded.keys())[0]

Upload your dataset CSV:


Saving Dataset-SA.csv to Dataset-SA.csv


In [16]:
def extract(filepath):
    df = pd.read_csv(filepath)
    print("Raw data shape:", df.shape)
    return df

In [17]:
def transform(df):
    df = df.copy()

    # Drop duplicates
    df = df.drop_duplicates()

    # Example: ensure numeric column
    if 'rating' in df.columns:
        df['rating'] = pd.to_numeric(df['rating'], errors='coerce')
        df['rating'] = df['rating'].fillna(df['rating'].median())

    # Example: clean text column
    if 'review' in df.columns:
        df['review'] = df['review'].astype(str).str.strip().str.lower()
        # Feature: length of review text
        df['review_length'] = df['review'].apply(lambda x: len(str(x)))

    # NumPy vectorized example: flag high ratings
    if 'rating' in df.columns:
        df['high_rating'] = np.where(df['rating'] >= 4, 1, 0)

    print("Transformed data shape:", df.shape)
    return df

In [18]:
def load(df, db_name='etl_pipeline.db', table_name='data'):
    engine = create_engine(f'sqlite:///{db_name}', echo=False)
    df.to_sql(table_name, con=engine, if_exists='replace', index=False)
    print(f"Loaded {len(df)} rows into table '{table_name}' in database '{db_name}'")
    return engine

In [19]:
def example_query(engine, table_name='data'):

    query = f"SELECT COUNT(*) AS n_rows FROM {table_name}"
    result = pd.read_sql(query, engine)
    print(result)

In [20]:
raw_df = extract(filename)
clean_df = transform(raw_df)
engine = load(clean_df, table_name='mytable')
example_query(engine, table_name='mytable')

# Download SQLite database back to your machine if you want
files.download('etl_pipeline.db')

Raw data shape: (205052, 6)
Transformed data shape: (170677, 6)
Loaded 170677 rows into table 'mytable' in database 'etl_pipeline.db'
   n_rows
0  170677


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [21]:
pd.read_sql("SELECT * FROM mytable LIMIT 5", engine)

,product_name,product_price,Rate,Review,Summary,Sentiment
0,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,5,super!,great cooler excellent air flow and for this p...,positive
1,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,5,awesome,best budget 2 fit cooler nice cooling,positive
2,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,3,fair,the quality is good but the power of air is de...,positive
3,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,1,useless product,very bad product its a only a fan,negative
4,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,3,fair,ok ok product,neutral


In [22]:
query1 = """
SELECT product_name, COUNT(*) AS n_reviews
FROM mytable
GROUP BY product_name
ORDER BY n_reviews DESC
LIMIT 10
"""
top_counts = pd.read_sql(query1, engine)
top_counts

,product_name,n_reviews
0,cello Pack of 18 Opalware Cello Dazzle Lush Fi...,4362
1,Lakm?? Eyeconic Kajal Twin Pack??????????(Deep...,2805
2,Mi 5A 80 cm (32 inch) HD Ready LED Smart Andro...,2122
3,Singer FM 1409 Electric Sewing MachineÐÒ?ÐÓ...,1743
4,"Canon EOS 3000D DSLR Camera 1 Camera Body, 18 ...",1646
5,cello Pack of 18 Opalware Cello Dazzle Lush Fi...,1642
6,NIVIA Storm Football - Size: 5ÐÓ®ÐÂ ÐÓ®ÐÂ ...,1613
7,"POCO C31 (Royal Blue, 64 GB)?ÐÒÐÒ?ÐÒÐÒ...",1569
8,DOMS Pencil Smart Kit,1545
9,Google Nest Mini (2nd Gen) with Google Assista...,1509


In [25]:
from sqlalchemy import inspect

insp = inspect(engine)
print(insp.get_columns('mytable'))

[{'name': 'product_name', 'type': TEXT(), 'nullable': True, 'default': None, 'primary_key': 0}, {'name': 'product_price', 'type': TEXT(), 'nullable': True, 'default': None, 'primary_key': 0}, {'name': 'Rate', 'type': TEXT(), 'nullable': True, 'default': None, 'primary_key': 0}, {'name': 'Review', 'type': TEXT(), 'nullable': True, 'default': None, 'primary_key': 0}, {'name': 'Summary', 'type': TEXT(), 'nullable': True, 'default': None, 'primary_key': 0}, {'name': 'Sentiment', 'type': TEXT(), 'nullable': True, 'default': None, 'primary_key': 0}]


In [27]:
query2 = """
SELECT product_name,
       ROUND(AVG(CAST(Rate AS FLOAT)), 2) AS avg_rating,
       COUNT(*) AS n_reviews
FROM mytable
GROUP BY product_name
HAVING COUNT(*) >= 5   -- only products with at least 5 reviews
ORDER BY avg_rating DESC
LIMIT 10
"""
avg_ratings = pd.read_sql(query2, engine)
avg_ratings

,product_name,avg_rating,n_reviews
0,ZunVolt Power MG5 500 Mixer Grinder 3 Jars White,5.00,10
1,Stylish Brown Sports Men Watches Multifunction...,5.00,5
2,LA VERNE Self Design Double Mink Blanket for ...,5.00,5
3,BAJAJ FX7 600 W Food Processor,5.00,10
4,Lopezs Microfiber Floor MatGrey23mm Medium,4.95,21
5,Educart CBSE Class 10 Sample Papers 202223 Bun...,4.94,16
6,"Viscose Blend Solid, Self Design Grey Men Dupatta",4.90,10
7,"Viscose Blend Solid, Self Design Dark Blue Men...",4.90,10
8,"Viscose Blend Solid, Self Design Black Men Dup...",4.90,10
9,"Viscose Blend Solid, Self Design Beige Men Dup...",4.90,10


In [28]:
query3 = """
SELECT
    CASE
        WHEN LENGTH(Review) < 50 THEN 'short'
        WHEN LENGTH(Review) < 200 THEN 'medium'
        ELSE 'long'
    END AS review_category,
    COUNT(*) AS count_reviews,
    ROUND(AVG(CAST(Rate AS FLOAT)),2) AS avg_rating
FROM mytable
GROUP BY review_category
ORDER BY count_reviews DESC
"""
review_length_stats = pd.read_sql(query3, engine)
review_length_stats

,review_category,count_reviews,avg_rating
0,short,154350,4.09
1,long,16202,3.81
2,medium,125,3.64


In [29]:
query5 = """
SELECT *
FROM mytable
WHERE CAST(Rate AS FLOAT) >= 4
LIMIT 10
"""
high_rating_reviews = pd.read_sql(query5, engine)
high_rating_reviews


,product_name,product_price,Rate,Review,Summary,Sentiment
0,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,5,super!,great cooler excellent air flow and for this p...,positive
1,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,5,awesome,best budget 2 fit cooler nice cooling,positive
2,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,5,awesome,the cooler is really fantastic and provides go...,positive
3,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,5,highly recommended,very good product,positive
4,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,4,worth the money,very good,positive
5,Candes 60 L Room/Personal Air Cooler??????(Whi...,8999,5,great product,beautiful product good material and perfectly ...,positive
6,Candes 60 L Room/Personal Air Cooler??????(Whi...,8999,5,mind-blowing purchase,awesome,positive
7,Candes 60 L Room/Personal Air Cooler??????(Whi...,8999,5,highly recommended,good,positive
8,Candes 60 L Room/Personal Air Cooler??????(Whi...,8999,5,brilliant,wonderful product must buy,positive
9,Candes 60 L Room/Personal Air Cooler??????(Whi...,8999,5,classy product,nice air cooler smart cool breeze producer,positive
